In [51]:
import numpy as np
import bokeh as bk
from bokeh.io import push_notebook
import bokeh.plotting as bkp
import bokeh.models as bkm
from bokeh.io import push_notebook
from ipywidgets import interact
import ipywidgets as widgets
bkp.output_notebook()

Loading BokehJS ...

# Analytical model for spherical sludge growth in Al-Fe system

## References:

1. An Analytical Solidification Model for Solute Redistribution during of Planar, Columnar, or Equiaxed Morphology L. Nastac and D.M. Stefanescu: Metall. Trans. A, 1993, vol. 24A, pp. 2107-18

2. Computational Modeling of NbC/Laves Formation In INCONEL 718 Equiaxed Castings L. Nastac and D.M. Stefanescu: Metall. Mater. Trans. A, 1997, vol. 28A, pp. 1582-1587

# Introduction:

## Goal: 

Application of Nastac-Stefanescu model for morphology of sludge ($ \beta $ aluminum) particles

## Assumptions:

<ul>
    <li>Instantaneous nucleation</li>
    <li>Sludge growth only in liquid</li>
    <li>negligible interference between growing
sludge</li>
    <li>volume diffusion limited growth of sludge.</li>
    <li>Constant growth velocity</li>
    
</ul>

## Schematic:

<img src="https://dev-plebbit.s3.amazonaws.com/uploads/topic/picture/22/nucleus.png" /img>

<strong> Figure 1. Schematic of volume element (from ref.1 ) </strong>

# Calculation strategy:

<ol>
    <li> Determine the material properties of Al </li>
    <li> Determine the material properties of Fe in Al </li>
    <li> Heat transfer: Nova </li>
    <li> Determine final solidification time </li>
    <li> Experimentally determine final sludge particle radius as function of cooling rate </li>
    <li> calculate interface growth velocity: </li>
    <li> $$ V_{\beta} = \frac{\Omega_{Fe}}{\Phi_{\beta}} \frac{D_L}{R_D-R_f} $$ </li>
    <li> $$ \Phi_{\beta} = \frac{4 \pi R_{\beta}}{A_s} $$ </li>
    <li> $$ R_D = \frac{3}{4\pi} \Bigg( \frac{v_l}{N_{\beta}} \Bigg)^{1/3} $$ </li>
    <li> $$ \Omega_{Fe} = \frac{C_L^* - C_0}{C_L^*-C_S^*} $$ </li>
    <li> Calculate interface radius: </li>
    <li> $$ R^* = V_{\beta}t $$ </li>
    <li> calculate f raction of solid: </li>
    <li> $$ f_s=(R^*/Rf)^3 $$</li>
    <li> Calculate interface liquid iron concentration as function of fraction of solid: </li>
    <li> $$ C_L^*= \frac{C_0\big( kI_S +I_L -\frac{1}{3} \big) + Q }{kI_S + I_L - \frac{1}{3} + \frac{1-k f_S}{3}} $$</li>
    <li> $$ Q= \frac{v_{\beta}}{v_{\Omega}}\frac{\rho_{\beta}}{\rho_{eu}} pct \; Fe_\beta  $$ </li>
    <li> $$ I_S = \frac{2f_S}{\pi^2} \sum_{n=1}^{\infty} \frac{1}{n^2} \Bigg[-\big( \frac{n \pi}{f_s^{1/3}} \big) \frac{D_St}{R^2_f} \Bigg]  $$ </li>
    <li> $$ I_L = 2f_s^{2/3}(1-f_s^{1/3}) \sum_{n=1}^{\infty} \frac{1}{\alpha^2_n} exp \Bigg[ -\Big( \frac{\alpha_n}{1/f_s^{1/3}} \Big)^2 \frac{D_L t}{R^2_f} \Bigg] $$ </li>
    <li> $$ \alpha_n R_f  \cot {\alpha_n} = R_f - R^* $$ </li>
    <li> Calculate sludge content (wt%) as function of fraction of solid </li>
</ol>

# Implementation:

## Experimental sludge size:

<img src="https://dev-plebbit.s3.amazonaws.com/uploads/topic/picture/24/sludge.png" />

# Heat transfer model (1D):

    1. A 383 as Al-Si system with Si-equivalent 
    2. Use chvorinov for primary phase
    3. Use kolmogorov for eutectic

$$ T^{n+1} = T^{n} - \frac{A}{v \rho_s} \frac{T^n-T_m}{\Delta H_f/\Delta T_0 + c_s} \sqrt{\frac{k_m \rho_m c_m}{\pi t^{n+1}}} \Delta t $$

Data taken from Nova database:

<table>
    <tr>
        <th>Material property</th>
        <td>$ \rho_s $</td>
        <td> $ c_s $ </td>
        <td> $ k_s $ </td>
        <td> $ v $</td>
        <td> $ A $</td>
        <td> $ \Delta H_{FP} $ </td>
        <td> $ \Delta H_{FE} $ </td>
        <td> $  T_L $ </td>
        <td> $  T_S $ </td>
        <td> $ \Delta T_0 $ </td>
        <td> $ T_E $ </td>
    </tr>
    <tr>
        <th>value</th>
        <td> 2485 </td>
        <td> 964.3 </td>
        <td> 96 </td>
        <td> 4.029×10^-5 m^3 </td>
        <td> 0.001089 m^2 </td>
        <td> 340 000 </td>
        <td> 508 497 </td>
        <td> 867 K </td>
        <td> 812 K </td>
        <td> 55 K </td>
        <td> 803 K </td>
    </tr>
</table>

Mold properties:

<table>
    <tr>
        <th>Material property</th>
        <td> $ \rho_m $ </td>
        <td> $ k_m $ </td>
        <td> $ c_m $ </td>
    </tr>
    <tr>
        <th>value</th>
        <td>1600</td>
        <td>0.52</td>
        <td>1170</td>
    </tr>
</table> 

In [52]:
Delta_t=1
T_i= 900
Delta_T_0=64
v=4.029*10**(-5)
A=0.001089
Delta_H_FP=340000
Delta_H_FE=508497
rho_s=2485
c_s=964
T_L=867
T_S=812
T_m=298.15
T_E=803

#cast iron mold:

rho_m=7050
k_m=40
c_m=794


#simulation parameters:

step_count=200

temp_values=np.empty(step_count)
temp_values[0]=T_i
    
def calc_temp(rho_m,k_m,c_m,temp_values):
    for i in range(1,step_count):
        if (temp_values[i-1]>T_L):
            temp_values[i]=temp_values[i-1]-A/v*(temp_values[i-1]-T_m)/(rho*(c))*np.sqrt((k_m*rho_m*c_m/(np.pi*(Delta_t*i))))*Delta_t
            
        elif (temp_values[i-1] > T_E):
            temp_values[i]=temp_values[i-1]-A/v*(temp_values[i-1]-T_m)/(rho*(Delta_H_FP/Delta_T_0 + c))*np.sqrt((k_m*rho_m*c_m/(np.pi*(Delta_t*i))))*Delta_t
            
        else:
            temp_values[i]=temp_values[i-1]-A/v*(temp_values[i-1]-T_m)/(rho*(Delta_H_FE/Delta_T_0 + c))*np.sqrt((k_m*rho_m*c_m/(np.pi*(Delta_t*i))))*Delta_t
            
calc_temp(rho_m,k_m,c_m,temp_values)
        
f=bkp.figure(title="Cooling curve of A 383 casting", x_axis_label="time (s)", y_axis_label="Temperature (K)")
time_series=np.linspace(0,step_count*Delta_t,step_count)
time_series_h=np.linspace(Delta_t,step_count*Delta_t,step_count)

f.line(time_series,temp_values, color="red", legend="Silica sand mold")

bkp.show(f)

## Cooling rate:

In [50]:
#Cooling rate, K/S
cooling_rate=np.array([1.2,1.7,2.8,10,25])
print(cooling_rate)

[  1.2   1.7   2.8  10.   25. ]


## Rf:

In [9]:
#Sludge area, um^2
sludge_area=np.array([380,290,240,235,245])
#Sludge area, m^2
sludge_area=sludge_area*10**(-12)
#Sludge radius, m
sludge_radius=np.sqrt(sludge_area)
print(sludge_radius)

[  1.94935887e-05   1.70293864e-05   1.54919334e-05   1.53297097e-05
   1.56524758e-05]


## Intermetallic growth:

In [25]:
cr=cooling_rate[0]
R_f=sludge_radius[0]
t_f=20
C_0=10.127
k=0.1539
N_beta=5
v_l=3/4*np.pi*R_f**3
D_L=2.69*10**(-9)
delta_t=1
pct_Fe_beta=18.15
rho_beta=2300
rho_eut=2300

R_D=(3/(4*np.pi))*(v_l/N_beta)**(1/3)


def sphere_volume(R):
    return (3/4*np.pi*R**3)

def sol_sup_sat(C_l):
    return (C_l-C_0)/(C_l-C_l/k)
    
def phi(R_beta,A_beta):
    return (4*np.pi*R_beta/A_beta)

def V_calc(C_l,R_beta,A_beta):
    return sol_sup_sat(C_l)/phi(R_beta,A_beta)*D_L/(R_D-R_beta)

def Q(R_beta):
    return sphere_colume(R_beta)/v_l*(rho_beta/rho_eut)

def I_S():

C_l=np.zeros(t_f)
V_beta=np.zeros(t_f)
R_beta=np.zeros(t_f)
A_beta=np.zeros(t_f)
f_s=np.zeros(t_f)

C_l[0]=C_0


V_beta[0]=V_calc(C_l[0],R_beta[0],A_beta[0])

R_beta[1]=R_beta[0]+V_beta[0]*delta_t
f_s[1]=(R_beta[1]/R_f)**3






    

for i in range(0,t_f,1):
    break
    
print(C_l_star)
    

IndentationError: expected an indented block (<ipython-input-25-bcbb1d9cfaa0>, line 34)